In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.transforms import ToTensor, Compose, Grayscale, Resize, CenterCrop
import matplotlib.pyplot as plt
from PIL import Image
from parse import parse
from custom_loader import AgeDBDataset
from custom_loss_functions import AngularPenaltySMLoss

import warnings
warnings.filterwarnings("ignore")

In [2]:
# hyper params
num_of_class = 79
hidden_unit = 256
learning_rate = 1e-04
batch_size = 20
device = torch.device("cuda")

In [3]:
dataset = AgeDBDataset(
    directory = 'AgeDB/',
    transform = Compose([
        Resize(size=(224, 224)),
        CenterCrop(size=224),
        Grayscale(num_output_channels=3),
        ToTensor(),
    ]),
    device = device,
)

In [4]:
train_set, validation_set, test_set = dataset.get_loaders(
    batch_size=batch_size,
    train_size=0.8,
    test_size=0.2,
)

In [5]:
def conv_layer(chann_in, chann_out, k_size, p_size):
    layer = nn.Sequential(
        nn.Conv2d(chann_in, chann_out, kernel_size=k_size, padding=p_size),
        nn.BatchNorm2d(chann_out),
        nn.ReLU()
    )
    return layer

def vgg_conv_block(in_list, out_list, k_list, p_list, pooling_k, pooling_s):

    layers = [ conv_layer(in_list[i], out_list[i], k_list[i], p_list[i]) for i in range(len(in_list)) ]
    layers += [ nn.MaxPool2d(kernel_size = pooling_k, stride = pooling_s)]
    return nn.Sequential(*layers)

def vgg_fc_layer(size_in, size_out):
    layer = nn.Sequential(
        nn.Linear(size_in, size_out),
        nn.BatchNorm1d(size_out),
        nn.ReLU()
    )
    return layer

class VGG16(nn.Module):
    def __init__(self, n_classes):
        super(VGG16, self).__init__()

        # Conv blocks (BatchNorm + ReLU activation added in each block)
        self.layer1 = vgg_conv_block([3,64], [64,64], [3,3], [1,1], 2, 2)
        self.layer2 = vgg_conv_block([64,128], [128,128], [3,3], [1,1], 2, 2)
        self.layer3 = vgg_conv_block([128,256,256], [256,256,256], [3,3,3], [1,1,1], 2, 2)
        self.layer4 = vgg_conv_block([256,512,512], [512,512,512], [3,3,3], [1,1,1], 2, 2)
        self.layer5 = vgg_conv_block([512,512,512], [512,512,512], [3,3,3], [1,1,1], 2, 2)

        # FC layers
        self.layer6 = vgg_fc_layer(7*7*512, 4096)
        self.layer7 = vgg_fc_layer(4096, 4096)

        # Final layer
        self.layer8 = nn.Linear(4096, n_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        vgg16_features = self.layer5(out)
        out = vgg16_features.view(out.size(0), -1)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)

        return vgg16_features, out

In [6]:
vgg = VGG16(n_classes=num_of_class).cuda()

In [7]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [8]:
# Training loop
def train(model, optimizer, criterion, train_loader, num_of_epoch):
    total_step = len(train_loader)
    for epoch in range(num_of_epoch):
        for i, (imgs, labels) in enumerate(train_loader):
            imgs = imgs.cuda()
            labels = torch.as_tensor(labels['age']).cuda()
            
            _, outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1)%total_step == 0:
                print(f"Epoch: {epoch+1}/{num_of_epoch}, Step: {i+1}/{total_step}, Loss: {loss.item()}")


In [9]:
train(vgg, optimizer, criteria, train_set, num_of_epoch=20)

Epoch: 1/20, Step: 621/621, Loss: 4.444716453552246
Epoch: 2/20, Step: 621/621, Loss: 4.202805995941162
Epoch: 3/20, Step: 621/621, Loss: 3.780808687210083
Epoch: 4/20, Step: 621/621, Loss: 3.52059268951416
Epoch: 5/20, Step: 621/621, Loss: 2.850433826446533
Epoch: 6/20, Step: 621/621, Loss: 1.4405899047851562
Epoch: 7/20, Step: 621/621, Loss: 0.38975122570991516
Epoch: 8/20, Step: 621/621, Loss: 0.2013775110244751
Epoch: 9/20, Step: 621/621, Loss: 0.8949185013771057
Epoch: 10/20, Step: 621/621, Loss: 0.05358529090881348
Epoch: 11/20, Step: 621/621, Loss: 0.09512737393379211
Epoch: 12/20, Step: 621/621, Loss: 0.05894378945231438
Epoch: 13/20, Step: 621/621, Loss: 0.04485530033707619
Epoch: 14/20, Step: 621/621, Loss: 0.009176983498036861
Epoch: 15/20, Step: 621/621, Loss: 0.056666482239961624
Epoch: 16/20, Step: 621/621, Loss: 0.13331864774227142
Epoch: 17/20, Step: 621/621, Loss: 0.009017972275614738
Epoch: 18/20, Step: 621/621, Loss: 0.01849314011633396
Epoch: 19/20, Step: 621/621, L

In [12]:
# Evaluation
def eval(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        error = torch.zeros(0).to(device)
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = torch.as_tensor(labels['age']).to(device)
            _, outputs = model(imgs)
            
            _, pred = torch.max(outputs.data, 1)

            error = torch.cat([error, torch.abs(
                torch.subtract(torch.reshape(labels, (-1,)), torch.reshape(pred, (-1,)))
            )])
            
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            
    print(f"Accuracy: {(100*correct)/total}%")
    print(f"Mean Absolute Error: {(torch.mean(error))}")
    print(f"Minimum: {torch.min(error)}, Maximum: {torch.max(error)}, Median: {torch.median(error)}")

In [13]:
eval(vgg, test_set)

Accuracy: 3.6428110896196%
Mean Absolute Error: 12.029980659484863
Minimum: 0.0, Maximum: 55.0, Median: 10.0


In [14]:
eval(vgg, train_set)

Accuracy: 35.29980657640232%
Mean Absolute Error: 8.027965545654297
Minimum: 0.0, Maximum: 58.0, Median: 4.0
